Install dependencies

In [ ]:
!pip install langchain langchain-community torch pypdf gpt4all faiss-gpu openai openpyxl docx beautifulsoup4 python-docx langchain-openai psycopg2-binary tiktoken


Install Ollama and pull model

In [ ]:
!curl https://ollama.ai/install.sh | sh

In [ ]:
!nohup ollama serve &

In [ ]:
!ollama pull llama2-uncensored

Vector database

In [ ]:
from  langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, CSVLoader
from langchain_community.vectorstores import FAISS
from  langchain_community.embeddings import GPT4AllEmbeddings


dataPath = '/content/drive/MyDrive/data'
vectorDBPath = 'stores/db_faiss'


class VectorDatabase:
    def __init__(self):
        pass

    def createFromFiles(self, dataPath, vectorDBPath):
        # Load data from files
        dirLoader = DirectoryLoader(dataPath, glob = '*.csv', loader_cls = CSVLoader)
        documents = dirLoader.load()

        # Split text into characters
        textSplitters = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
        chunks = textSplitters.split_documents(documents)

        # Generate embeddings
        embeddings = GPT4AllEmbeddings(model_file = 'models/all-MiniLM-L6-v2')
        db = FAISS.from_documents(chunks, embeddings)
        db.save_local(vectorDBPath)

        return db

    def createFromDir(self, dataPath, vectorDBPath):
        # Load data from files
        dirLoader = DirectoryLoader(dataPath)
        documents = dirLoader.load()

        # Split text into characters
        textSplitters = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
        chunks = textSplitters.split_documents(documents)

        # Generate embeddings
        embeddings = GPT4AllEmbeddings(model_file = 'models/all-MiniLM-L6-v2')
        db = FAISS.from_documents(chunks, embeddings)
        db.save_local(vectorDBPath)

        return db



vectorDb = VectorDatabase()
vectorDb.createFromFiles(dataPath = dataPath, vectorDBPath = vectorDBPath)

Downloading: 100%|██████████| 45.9M/45.9M [00:00<00:00, 79.5MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 553MiB/s]


Application Chain

In [ ]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from  langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import FAISS


# Load LLM

class AppChain:
    def __init__(self):
        pass

    def loadModel(self, model):
        llm = Ollama(
            model = model,
            callback_manager=CallbackManager(
                [StreamingStdOutCallbackHandler()],
            ),
        )

        return llm

    def createPrompt(self, context, template):
        prompt = PromptTemplate(template = template, input_variables = ["context", "question"])

        return prompt

    def createChain(self, prompt, llm, db):
        llmChain = RetrievalQA.from_chain_type(
            llm = llm,
            chain_type = 'stuff',
            chain_type_kwargs = {'prompt' : prompt},
            retriever = db.as_retriever(search_kwargs = {'k': 3}, max_new_tokens = 1024),
            return_source_documents = False,
        )

        return llmChain

    def readVectorDB(self, vectorDBPath, model):

        # Embeddings
        embedding = GPT4AllEmbeddings(model_file = model)
        db = FAISS.load_local(vectorDBPath, embedding, allow_dangerous_deserialization = True)

        return db


# Load chain and model

chain = AppChain()
llm = chain.loadModel("llama2-uncensored")
db = chain.readVectorDB('stores/db_faiss', 'models/all-MiniLM-L6-v2')


# Geneate prompt
promtTemplate = """<|im_start|>system
se the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
<|im_end|>
<|im_start|>user
{question}
<|im_end|>
<|im_start|>assistant
"""

# Create prompt
prompt = chain.createPrompt('context', promtTemplate)

# Run chain
llmChain = chain.createChain(prompt, llm, db)
reponse = llmChain.invoke({'query': 'Explain about re-route ?'})
print(reponse)